In [ ]:
# Code modified from AllOfUs workbench notebooks inc. https://workbench.researchallofus.org/workspaces/aou-rw-ec29d5cc/howtorunwdlsusingcromwellintheresearcherworkbench/notebooks/preview/Validate%20VCFs%20with%20Cromwell.ipynb
!curl https://github.com/broadinstitute/cromwell/releases/download/77/cromwell-77.jar -o cromwell-77.jar -L
!curl https://github.com/broadinstitute/cromwell/releases/download/77/womtool-77.jar -o womtool-77.jar -L
!curl -s "https://get.sdkman.io" -o install_sdkman.sh
!bash install_sdkman.sh

In [ ]:
import os 
bucket = os.getenv("WORKSPACE_BUCKET")
project = os.getenv("GOOGLE_PROJECT")
 
cromwell_test_workdir = bucket + "/"  # Later, "cromwell-executions" will be appended to this for cromwell-workflow storage.
output_bucket = bucket + "/test_cromwell_runs"  # This is where the output of the WDL will be.
 
print(f'Workspace bucket: {bucket}')
print(f'Workspace project: {project}')
print(f'Workspace cromwell working bucket: {cromwell_test_workdir}')
print(f'Workspace output bucket: {output_bucket}')

In [ ]:
service_account = !gcloud auth list --format json | jq -r .[0].account
sa = service_account[0]
print(f'Pet service account Email: {sa}')

In [ ]:
json_filename = "validate_vcf.json"

#fill in or leave empty as needed for the json creation here
json_content = """{
  
}

"""

# "validate_vcf.ordered_vcf_shards_idx_list": "vcf_tbi_file_list.txt",
#   "validate_vcf.ref_fasta": "gs://genomics-public-data/references/hg38/v0/Homo_sapiens_assembly38.fasta",
#   "validate_vcf.ref_fai": "gs://genomics-public-data/references/hg38/v0/Homo_sapiens_assembly38.fasta.fai",
#   "validate_vcf.ref_dict": "gs://genomics-public-data/references/hg38/v0/Homo_sapiens_assembly38.dict",
#   "validate_vcf.ordered_vcf_shards_list": "vcf_file_list.txt"

fp = open(json_filename, 'w')
fp.write(json_content)
fp.close()
#print(json_content)

In [ ]:
# options - need to specify the bucket path for cromwell - don't change

options_filename = "options.json"

options_content = f'{{\n  "jes_gcs_root": "{output_bucket}"\n}}'

fp = open(options_filename, 'w')
fp.write(options_content)
fp.close()

In [ ]:
#!source "/home/jupyter/.sdkman/bin/sdkman-init.sh"
#!sdk install java 11.0.14-tem
#validate wdl
#!java -jar womtool-77.jar validate _WDL_FILE_
cromwell_run_cmd = 'source "/home/jupyter/.sdkman/bin/sdkman-init.sh" &&  sdk install java 11.0.14-tem && echo "Validating WDL..." && java -jar womtool-77.jar validate _WDL_FILE_ && java -jar -Dconfig.file=/home/jupyter/cromwell.conf cromwell-77.jar run _WDL_FILE_ -i _JSON_FILE_ -o _OPTIONS_FILE_'

#run wdl
#!java -jar -Dconfig.file=test_PAPIv2.conf cromwell-77.jar run _WDL_FILE_ -i _JSON_FILE_ -o _OPTIONS_FILE_

cromwell_run_cmd_final = cromwell_run_cmd.replace("_WDL_FILE_", wdl_filename)
cromwell_run_cmd_final = cromwell_run_cmd_final.replace("_JSON_FILE_", json_filename)
cromwell_run_cmd_final = cromwell_run_cmd_final.replace("_OPTIONS_FILE_", options_filename)
print(cromwell_run_cmd_final)




In [ ]:
## output example from workflow at end of cromwell log msg
# "outputs": {
#     "wf_hello.hello.message": "gs://fc-secure-2e7bdd78-3d9f-4cf7-b5e7-9d4ea20c743e/test_cromwell_runs/wf_hello/f890c19e-ef96-445c-8a9d-8542a323fa1b/call-hello/message.txt"
#   },


##validate task output examples:
# workflow_name = "validate_vcf"  # found in the wdl and cromwell log message
# workflow_id = "f890c19e-ef96-445c-8a9d-8542a323fa1b" # found in the cromwell logs, listed as "id"
# task_name = "validate"
# shard = "shard-0" 

# task_call = "call-" + task_name
# ! gsutil ls {output_bucket}/{workflow_name}/{workflow_id}/{task_call}/{shard}/